# Lessons from gridsearch

No dropout

No warmup



In [9]:
import pandas as _pd
import torch as _torch
import numpy as _np
import torch.utils.data as _torchdata

if __package__ is None or __package__ == '':
    import vambtools as _vambtools
    
else:
    import vamb.vambtools as _vambtools

ModuleNotFoundError: No module named 'torch.Autograd'

In [1]:
def load(depthspath, tnfpath, use_cuda, batchsize, shuffle):
    depths = _load_tsv(depthspath)
    tnf = _load_tsv(tnfpath)
    
    dataset = TensorDataset(data_tensor, tnf_tensor)
   
    return _torchdata.DataLoader(dataset=dataset, batch_size=batchsize, shuffle=shuffle,
                          num_workers=1, pin_memory=use_cuda)

In [ ]:
class VAE(_torch.nn.Module):
    """Variational autoencoder"""
    
    def __init__(self, nsamples, nhiddens, nlatent):
        if len(nhiddens) < 1:
            raise ValueError('Must have at least one hidden layer')
        
        super(VAE, self).__init__()
        
        # Initialize normal attributes
        self.nsamples = nsamples
        self.nfeatures = self.nsamples + 136
        self.nhiddens = nhiddens
        self.nlatent = nlatent
        
        # Lists for holding encoder and decoder hidden layers
        self.encoder_hidden_linears = _torch.nn.ModuleList()
        self.decoder_hidden_linears = _torch.nn.ModuleList()
        
        self.encoder_hidden_batchnorms = _torch.nn.ModuleList()
        self.decoder_hidden_batchnorms = _torch.nn.ModuleList()
        
        # Add first encoding hidden layer
        first_linear = _torch.nn.Linear(self.nfeatures, self.nhiddens[0])
        first_batchnorm = _torch.nn.BatchNorm1d(self.nhiddens[0])
        self.encoder_hidden_linears.append(first_linear)
        self.encoder_hidden_batchnorms.append(first_batchnorm)
        
        # Add the next hidden layers, if any
        for ninput, noutput in zip(self.nhiddens, self.nhiddens[1:]):
            linear = _torch.nn.Linear(ninput, noutput)
            batchnorm = _torch.nn.BatchNorm1d(noutput)
            
            self.encoder_hidden_linears.append(linear)
            self.encoder_hidden_batchnorms.append(batchnorm)
            
        # Latent layers
        self.mu = _torch.nn.Linear(self.nhiddens[-1], self.nlatent)
        self.logsigma = _torch.nn.Linear(self.nhiddens[-1], self.nlatent)
        
        # Now add the decoding layers
        first_linear = _torch.nn.Linear(self.nlatent, self.nhiddens[-1])
        first_batchnorm = _torch.nn.BatchNorm1d(self.nhiddens[-1])
        self.decoder_hidden_linears.append(first_linear)
        self.decoder_hidden_batchnorms.append(first_batchnorm)
        
        for ninput, noutput in zip(self.nhiddens[-1, -1, -1], self.nhiddens[-2, -1, -1]):
            linear = _torch.nn.Linear(ninput, noutput)
            batchnorm = _torch.nn.BatchNorm1d(noutput)
            
            self.decoder_hidden_linears.append(linear)
            self.decoder_hidden_batchnorms.append(batchnorm)
            
        # Final output layer
        self.reconstructed = _torch.nn.Linear(self.nhidden[0], self.nfeatures)

        # Activator functions (these are layers)
        self.relu = _torch.nn.LeakyReLU()
        self.softplus = _torch.nn.Softplus()
        
    def encode(self, data):
        for linear, batchnorm in zip(self.encoder_hidden_linears, self.encoder_hidden_batchnorms):
            data = linear(data)
            data = self.relu(data)
            data = batchnorm(data)
            
        mu = self.mu(data)
        logsigma = self.logsigma(data)
        logsigma = self.softplus(logsigma)
        
        return mu, logsigma
    
    def reparameterize(self, mu, logsigma, usecuda):
        epsilon = _torch.randn(*mu.size)
        
        if usecuda:
            epsilon = epsilon.cuda()
        
        epsilon = _torch.autograd.Variable(epsilon)
        
        latent = mu + epsilon * _torch.exp(logsigma / 2)
        
        return latent
    
    def decode(self, latent):
        for linear, batchnorm in zip(self.decoder_hidden_linears, self.decoder_hidden_batchnorms):
            data = linear(data)
            data = self.relu(data)
            data = batchnorm(data)
            
        latent_both = self.reconstructed(data)
        
        reconstructed_depths = _torch.nn.functional.softmax(latent_both.narrow(1, 0, self.nsamples), dim=1)
        reconstructed_tnf = latent_both.narrow(1, self.nsamples, 136)
    
        return reconstructed_depths, reconstructed_tnf
    
    def forward(self, depths, tnf):
        data = _torch.cat((depths, tnf), 1)
        mu, logsigma = self.encode(data)
        latent = self.reparameterize(mu, logsigma)
        reconstructed_depths, reconstructed_tnf = self.decode(latent)
        
        return reconstructed_depths, reconstructed_tnf, mu, logsigma

In [8]:
torch.Autograd

AttributeError: module 'torch' has no attribute 'Autograd'

In [11]:
def calculate_loss(depths, depths_rec, tnf, tnf_rec, mu, logsigma, tnf_factor):
    MSE = _torch.nn.MSELoss() 
    mse = _torch.nn.MSELoss(tnf_rec, tnf)
    bce = _torch.nn.functional.binary_cross_entropy(depths_rec, depths, size_average=False)
    kld = -0.5 * _torch.mean(1 + logsigma - mu.pow(2) - logsigma.exp())
    loss = bce + kld + tnf_factor * mse
    
    return loss, bce, mse, kld

In [ ]:
def train_epoch(model, epoch, dataloader, optimizer, tnf_factor, usecuda):
    model.train()
    
    epoch_loss = 0
    epoch_bce = 0
    epoch_mse = 0
    epoch_kld = 0
    
    # Get mini batches
    for nbatch, (depths, tnf) in enumerate(dataloader):
        # Get the data
        depths = _torch.autograd.Variable(depths)
        tnf = _torch.autograd.Variable(tnf)
        
        if usecuda:
            depths = depths.cuda()
            tnf = tnf.cuda()
        
        # Do not carry over from previous mini batch
        optimizer.zero_grad()
        
        # Do a forward pass
        depths_rec, tnf_rec, mu, logsigma = model(depths, tnf)
        loss, bce, mse, kld = calculate_loss(depths, depths_rec, tnf, tnf_rec, mu, logsigma, tnf_factor)
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        # Keep track of per-epoch statistics
        epoch_loss += loss.data[0]
        epoch_bce += bce.data[0]
        epoch_mse += mse.data[0]
        epoch_kld += kld.data[0]
        
        # Print per-100 minibatches statistics
        if nbatch % 100 == 0:
            message = '\t{:{width}} / {}:\tLoss: {:.4f}\tBCE: {:.4f}\tMSE: {:.4f}\tKLD: {:.4f}'.format(
                       nbatch * len(depths),
                       len(dataloader.dataset),
                       loss.data[0] / len(depths),
                       bce.data[0] / len(depths),
                       mse.data[0] / len(depths),
                       kld.data[0] / len(depths),
                       width=len(str(len(dataloader.dataset))))
            
            print(message)
            
    # Print epoch statistics
    message = '\nEpoch {}:\tLoss: {:.4f}\tBCE: {:.4f}\tMSE: {:.4f}\tKLD: {:.4f}'.format(
           epoch,
           loss.data[0] / len(depths),
           bce.data[0] / len(depths),
           mse.data[0] / len(depths),
           kld.data[0] / len(depths))
    
    print(message)

In [ ]:
def main(args):
    

In [ ]:
def main(args):
   '''Main program'''
   
   # get number of features
   n_features = []
   n_features.append(get_features(args.i[0]))
   n_features.append(get_features(args.tnf[0]))
   
   # load data for training
   if args.to_memory:
      data_loader = load_data_mem(args)
   else:
      data_loader = load_data(args)
   
   # define model and optimizer
   model = VAE(n_features, args.dp_rate)
   if args.cuda:
      model.cuda()
   optimizer = optim.Adam(model.parameters(), lr=args.lrate)
   
   # set beta
   if args.warmup == 0:
      beta = 1
      to_add = 0
   else:
      to_add = 1/args.warmup
      beta = 0
   
   # train
   for epoch in range(1, args.nepochs + 1):
      train(epoch, model, data_loader, optimizer, beta, args.mse)
      if beta >= 1:
         beta = 1
      else:
         beta = beta + to_add
   
   # evaluate
   # currently data does not need to be reloaded because input-data is already shuffled #
   # load data for eval
   #if args.to_memory:
   #   data_loader_eval = load_data_mem(args, training=False)
   #else:
   #   data_loader_eval = load_data(args, training=False)
   #evaluate(model, data_loader_eval, beta, args.o)
   evaluate(model, data_loader, beta, args.mse, args.o)


In [17]:
a = 5
'{:4}'.format(5)

'   5'

In [ ]:
def train(epoch, model, data_loader, optimizer, beta, mse):
    model.train()
    train_loss = 0
    KLD_loss = 0
    BCE_loss = 0
    MSE_loss = 0
    for batch_idx, (data,tnf) in enumerate(data_loader):
        data = Variable(data)
        tnf = Variable(tnf)
        if args.cuda:
            data = data.cuda()
            tnf = tnf.cuda()
        optimizer.zero_grad()
        y_data, y_tnf, mu, logvar = model(data, tnf)
        BCE, KLD, loss, MSE = loss_function(y_data, data, y_tnf, tnf, mu, logvar, beta, mse)
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0]
        KLD_loss += KLD.data[0]
        BCE_loss += BCE.data[0]
        MSE_loss += MSE.data[0]
        if batch_idx % 50 == 0:
         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}\tBCE: {:.4f}\tKLD: {:.4f}\tBeta: {:.2f}'.format(
            epoch, batch_idx * len(data), len(data_loader.dataset),
            100. * batch_idx / len(data_loader),
            loss.data[0] / len(data),
            BCE.data[0] / len(data),
            KLD.data[0] / len(data),
            beta))

    print('====> Epoch: {}\tLoss: {:.4f}\tBCE: {:.4f}\tKLD: {:.4f}\tMSE: {:.5f}\tBeta: {:.2f}'.format(
          epoch, train_loss / len(data_loader.dataset),
          BCE_loss / len(data_loader.dataset),
          KLD_loss / len(data_loader.dataset),
          MSE_loss / len(data_loader.dataset),
          beta))

In [7]:
_torch.nn.functional.binary_cross_entropy

<function torch.nn.functional.binary_cross_entropy(input, target, weight=None, size_average=True)>

In [60]:
def _load_tsv(path, skiprows=1, skipcols=1):
    with _vambtools.Reader(path) as filehandle:
        firstline = next(filehandle)
    
    ncolumns = len(firstline.split('\t'))
    
    if skipcols >= ncolumns:
        raise ValueError('Cannot skip {} of {} columns in {}'.format(
            skipcols, ncolumns, path))
        
    columns = list(range(skipcols, ncolumns))
    
    dataframe = _pd.read_csv(path, delimiter='\t', header=None,
                        skiprows=skiprows, usecols=columns, dtype=_np.float32)
    
    return _torch.from_numpy(dataframe.values)

In [61]:
def load(path)


 0.4182  0.2403  0.1667  0.0971  0.0777
 0.3748  0.2163  0.1910  0.1362  0.0816
 0.3861  0.2911  0.1629  0.0910  0.0689
                   ⋮                    
 0.2300  0.2188  0.1409  0.1794  0.2309
 0.1235  0.3253  0.2218  0.1845  0.1449
 0.2790  0.2013  0.1419  0.0808  0.2970
[torch.FloatTensor of size 275285x5]

In [ ]:
def load_data_mem(args, training=True):
   '''Data loader to memory'''
   
   data_tensor = torch.Tensor(np.asarray(pd.read_csv(args.i[0], delim_whitespace=True, header=None)))
   tnf_tensor = torch.Tensor(np.asarray(pd.read_csv(args.tnf[0], delim_whitespace=True, header=None)))
   dataset = TensorDataset(data_tensor, tnf_tensor)
   
   if args.cuda:
      kwargs = {'num_workers': 1, 'pin_memory': True}
   else:
      kwargs = {'num_workers': 1, 'pin_memory': False}
   
   if training:
      data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch, shuffle=False, **kwargs)
   else:
      data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch, shuffle=False, **kwargs)
   return data_loader